We could also have called this topic "v-lookups" because they are used in Excel to "join" separate tables.  We will cover the theory as well in case the reader is not familiar with the database operations we are replicating